# Program Repair

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8" # this is needed to get rid of weird colab locale error
# if you are still running into issues, please restart the runtime to initialize a new environment

In [4]:
# installing the accelerate library
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [5]:
# https://github.com/evalplus/evalplus
!pip install evalplus==0.2.0

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tempdir: filename=tempdir-0.7.1-py3-none-any.whl size=2196 sha256=6b50f4735df6f4535ffe9967b1aafde49d69fb7ae5c82a4c37d6443f3c607460
  Stored in directory: /root/.cache/pip/wheels/12/67/0b/519393cca63cd89cea554a371cc88a4c439c2d31804e9a9ed7
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=2b08c8f30f6d5fa791bf4df3ed2c3414882d2808de60ec6af7441ee7c34474f2
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built tempdir wget


In [6]:
!wget https://raw.githubusercontent.com/uiuc-cs598lmz-s25/hw3/main/buggy_humaneval.jsonl

--2025-03-28 17:48:22--  https://raw.githubusercontent.com/uiuc-cs598lmz-s25/hw3/main/buggy_humaneval.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117423 (115K) [text/plain]
Saving to: ‘buggy_humaneval.jsonl’

buggy_humaneval.jso 100%[===================>] 114.67K  --.-KB/s    in 0.003s  

2025-03-28 17:48:23 (41.9 MB/s) - ‘buggy_humaneval.jsonl’ saved [117423/117423]



In [7]:
import json

def grab_buggy_dataset():
    inference_dataset = []
    file = "buggy_humaneval.jsonl"
    with open(file, "r") as f:
        inference_dataset.extend([json.loads(x) for x in f.readlines()])
    print("Number of tasks: {}".format(len(inference_dataset)))
    return inference_dataset

buggy_humaneval = grab_buggy_dataset()
# feel free to play around the dataset for a bit

Number of tasks: 164


In [8]:
import torch

from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM, T5ForConditionalGeneration

def load_codegen_model():
    tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
    model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-2B-mono", device_map='auto', torch_dtype=torch.float16)
    return model, tokenizer

def load_codet5_model():
    tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")
    model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base", device_map='auto')
    return model, tokenizer

codegen, codegen_tokenizer = load_codegen_model()
codet5, codet5_tokenizer = load_codet5_model()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [9]:
# make the folder to save the results
!mkdir codegen_fix

In [11]:
import os
from tqdm.auto import tqdm
import re

def codegen_program_repair(buggy_program: str, buggy_line_index: int, model, tokenizer) -> str:
    """
    Uses CodeGen to repair a single buggy line in a program, ensuring proper indentation.

    Args:
        buggy_program (str): The complete function with a buggy line.
        buggy_line_index (int): The index (zero-indexed) of the buggy line in the program.
        model: The pretrained CodeGen model.
        tokenizer: The tokenizer associated with the CodeGen model.

    Returns:
        str: The complete function with the buggy line replaced by a generated patch.
    """
    # Split the program into lines.
    lines = buggy_program.split("\n")
    # The prefix is all lines before the buggy line.
    prefix = "\n".join(lines[:buggy_line_index])

    # Tokenize the prefix and move tensors to the model's device.
    inputs = tokenizer(prefix, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    # Generate new tokens using greedy decoding.
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,
            do_sample=False,  # Greedy decoding; temperature not needed.
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated tokens.
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Remove the prefix from the generated text to isolate the patch.
    patch_text = generated_text[len(prefix):].lstrip()
    # Extract the first generated line as the patch.
    patch_line_raw = patch_text.split("\n")[0].strip()

    # Get the original indentation of the buggy line.
    original_line = lines[buggy_line_index]
    indent_match = re.match(r"\s*", original_line)
    original_indent = indent_match.group(0) if indent_match else ""

    # Prepend the original indentation to the generated patch line.
    patch_line = original_indent + patch_line_raw

    # Replace the buggy line with the new patch.
    lines[buggy_line_index] = patch_line

    # Reconstruct and return the complete function.
    repaired_program = "\n".join(lines)
    return repaired_program


def repair_humaneval_codegen(model, tokenizer, buggy_humaneval, workdir):
  for bug in tqdm(buggy_humaneval):
      name = bug['task_id'].replace("/", "_")
      buggy_code = bug['buggy_code']
      buggy_line_index = bug['buggy_line']

      fixed_code = codegen_program_repair(buggy_code, buggy_line_index, model, tokenizer)
      os.makedirs(os.path.join(workdir, name), exist_ok=True)
      with open(os.path.join(workdir, name, '0.py'), 'w') as f:
          f.write(fixed_code)


# generate the solutions produced by codegen
repair_humaneval_codegen(codegen, codegen_tokenizer, buggy_humaneval, "codegen_fix")

  0%|          | 0/164 [00:00<?, ?it/s]

In [17]:
# now place take a look at the solutions produced by codegen in the folder
# we will now evaluate the solution
# note you can passing in "--i-just-wanna-run" to this command to
# recompute the results IF and ONLY IF you have made some updates to each solution file :)
!yes Y | evalplus.evaluate --dataset humaneval --samples codegen_fix --i-just-wanna-run

Load from ground-truth from /root/.cache/evalplus/84f4b93a1270b492e4c54d5212da7a5b.pkl
Reading samples...
164it [00:00, 2788.03it/s]
100% 164/164 [00:21<00:00,  7.71it/s]
Base
{'pass@1': np.float64(0.40853658536585363)}
Base + Extra
{'pass@1': np.float64(0.3902439024390244)}


In [12]:
# make the folder to save the results
!mkdir codet5_fix

mkdir: cannot create directory ‘codet5_fix’: File exists


In [13]:
import os
from tqdm.auto import tqdm

def codet5_program_repair(buggy_program: str, buggy_line_index: int, model, tokenizer) -> str:
    """
    Uses CodeT5 to repair a single buggy line in a program, ensuring proper indentation.
    This version uses both the prefix (code before the buggy line) and suffix (code after the buggy line)
    as context. The input prompt is constructed by placing a placeholder <extra_id_0> where the buggy line is.
    The model then fills in this placeholder with a candidate patch.

    Args:
        buggy_program (str): The complete function with a buggy line.
        buggy_line_index (int): The index (zero-indexed) of the buggy line in the program.
        model: The pretrained CodeT5 model.
        tokenizer: The tokenizer associated with the CodeT5 model.

    Returns:
        str: The complete function with the buggy line replaced by a generated patch.
    """
    # Split the program into individual lines.
    lines = buggy_program.split("\n")

    # Extract prefix (lines before the buggy line) and suffix (lines after the buggy line).
    prefix = "\n".join(lines[:buggy_line_index])
    suffix = "\n".join(lines[buggy_line_index+1:])

    # Construct the input prompt using <extra_id_0> as a placeholder for the buggy line.
    input_prompt = prefix + "\n<extra_id_0>\n" + suffix

    # Tokenize the input prompt and move tensors to the model's device.
    inputs = tokenizer(input_prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    # Generate new tokens using greedy decoding.
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,
            do_sample=False,  # Greedy decoding
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated tokens (keep special tokens to help locate the placeholder boundaries).
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

    # Extract the candidate patch from the generated text.
    # Preferably, the model fills the gap between <extra_id_0> and <extra_id_1>.
    match = re.search(r"<extra_id_0>(.*?)<extra_id_1>", generated_text, re.DOTALL)
    if match:
        patch_line_raw = match.group(1).strip()
    else:
        # Fallback: if <extra_id_1> is not found, try to use text after <extra_id_0>.
        if "<extra_id_0>" in generated_text:
            patch_line_raw = generated_text.split("<extra_id_0>")[-1].strip().split("\n")[0].strip()
        else:
            patch_line_raw = generated_text.split("\n")[0].strip()

    # Retrieve the original indentation from the buggy line.
    original_line = lines[buggy_line_index]
    indent_match = re.match(r"\s*", original_line)
    original_indent = indent_match.group(0) if indent_match else ""

    # Prepend the original indentation to the generated patch.
    patch_line = original_indent + patch_line_raw

    # Replace the buggy line with the new patch.
    lines[buggy_line_index] = patch_line

    # Reconstruct and return the complete function.
    repaired_program = "\n".join(lines)
    return repaired_program


def repair_humaneval_codet5(model, tokenizer, buggy_humaneval, workdir):
  for bug in tqdm(buggy_humaneval):
      name = bug['task_id'].replace("/", "_")
      buggy_code = bug['buggy_code']
      buggy_line_index = bug['buggy_line']

      fixed_code = codet5_program_repair(buggy_code, buggy_line_index, model, tokenizer)
      os.makedirs(os.path.join(workdir, name), exist_ok=True)
      with open(os.path.join(workdir, name, '0.py'), 'w') as f:
          f.write(fixed_code)


repair_humaneval_codet5(codet5, codet5_tokenizer, buggy_humaneval, "codet5_fix")

  0%|          | 0/164 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (576 > 512). Running this sequence through the model will result in indexing errors


In [14]:
# now place take a look at the solutions produced by codegen in the folder
# we will now evaluate the solution
# note you can passing in "--i-just-wanna-run" to this command to
# recompute the results IF and ONLY IF you have made some updates to each solution file :)
!yes Y | evalplus.evaluate --dataset humaneval --samples codet5_fix --i-just-wanna-run

Load from ground-truth from /root/.cache/evalplus/84f4b93a1270b492e4c54d5212da7a5b.pkl
Reading samples...
164it [00:00, 1997.94it/s]
100% 164/164 [00:10<00:00, 15.09it/s]
Base
{'pass@1': np.float64(0.054878048780487805)}
Base + Extra
{'pass@1': np.float64(0.054878048780487805)}


In [15]:
from evalplus.data import get_human_eval_plus

def check_which_passed(workdir: str, dataset):
    with open(os.path.join(workdir, "eval_results.json"), "r") as f:
        results = json.loads(f.read())

    failed_humaneval = []
    failed_humaneval_plus = []

    for task_id in dataset.keys():
        total = results['eval'][task_id]['nfiles']
        humaneval_base = len([x for x in results['eval'][task_id]['base'] if x[0] == "success"]) / total
        humaneval_plus = len([x for x in results['eval'][task_id]['plus'] if x[0] == "success"]) / total

        if humaneval_base == 1:
            failed_humaneval.append(task_id)
        if humaneval_plus == 1:
            failed_humaneval_plus.append(task_id)

    return failed_humaneval, failed_humaneval_plus


In [18]:
# you can use this to check which problem the model fixed
success_humaneval, success_humaneval_plus = check_which_passed("codegen_fix", get_human_eval_plus())

In [19]:
success_humaneval_codet5, success_humaneval_plus_codet5 = check_which_passed("codet5_fix", get_human_eval_plus())

# Template-based Program Repair

In [20]:
# make the folder to save the results
!cp -r codet5_fix codet5_fix_template

In [24]:
import keyword

def template_codet5_program_repair(buggy_program: str, buggy_line_index: int, model, tokenizer) -> str:
    """
    Uses CodeT5 to repair a single buggy line by applying a variable name template.
    The function identifies a candidate variable name in the buggy line, replaces it with a placeholder
    (<extra_id_0>), and then uses CodeT5 to generate the correct variable name. The generated name is then
    substituted back into the line while preserving the original indentation.

    Args:
        buggy_program (str): The complete function with a buggy line.
        buggy_line_index (int): The index (zero-indexed) of the buggy line in the program.
        model: The pretrained CodeT5 model.
        tokenizer: The tokenizer associated with the CodeT5 model.

    Returns:
        str: The complete function with the buggy line repaired.
    """
    # Split the program into lines and extract the buggy line.
    lines = buggy_program.split("\n")
    original_line = lines[buggy_line_index]

    # Identify candidate variable names using a regex.
    # We match typical Python identifiers.
    var_candidates = re.findall(r"\b[a-zA-Z_][a-zA-Z0-9_]*\b", original_line)
    # Filter out Python keywords and built-in names.
    var_candidates = [var for var in var_candidates if not keyword.iskeyword(var) and var not in dir(__builtins__)]

    if var_candidates:
        # Choose the first candidate variable as the one to repair.
        candidate_var = var_candidates[0]
        # Create a template by replacing the candidate variable with the placeholder.
        template_line = original_line.replace(candidate_var, "<extra_id_0>", 1)
    else:
        # Fallback: if no candidate variable is found, use a generic placeholder.
        template_line = "<extra_id_0>"

    # Construct a prompt instructing CodeT5 to replace the placeholder with the correct variable name.
    input_prompt = (
        "Replace <extra_id_0> with the correct variable name in the following line:\n" +
        template_line
    )

    # Tokenize the prompt and move tensors to the model's device.
    inputs = tokenizer(input_prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    # Generate new tokens using greedy decoding.
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated tokens.
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

    # Extract the generated variable name.
    match_gen = re.search(r"<extra_id_0>(.*?)<extra_id_1>", generated_text, re.DOTALL)
    if match_gen:
        replacement_var = match_gen.group(1).strip()
    else:
        # Fallback extraction if the expected markers are absent.
        if "<extra_id_0>" in generated_text:
            replacement_var = generated_text.split("<extra_id_0>")[-1].strip().split()[0]
        else:
            replacement_var = generated_text.strip().split()[0]

    # Replace the placeholder in the template line with the generated variable name.
    repaired_line_raw = template_line.replace("<extra_id_0>", replacement_var)

    # Retrieve the original indentation from the buggy line.
    indent_match = re.match(r"\s*", original_line)
    original_indent = indent_match.group(0) if indent_match else ""
    repaired_line = original_indent + repaired_line_raw.lstrip()

    # Replace the buggy line in the program with the repaired line.
    lines[buggy_line_index] = repaired_line
    repaired_program = "\n".join(lines)
    return repaired_program


def repair_humaneval_codet5_template(model, tokenizer, buggy_humaneval, fixed_bugs, workdir):
  for bug in tqdm(buggy_humaneval):
      name = bug['task_id'].replace("/", "_")
      if bug['task_id'] in fixed_bugs:
        continue
      buggy_code = bug['buggy_code']
      buggy_line_index = bug['buggy_line']

      fixed_code = template_codet5_program_repair(buggy_code, buggy_line_index, model, tokenizer)
      os.makedirs(os.path.join(workdir, name), exist_ok=True)
      with open(os.path.join(workdir, name, '0.py'), 'w') as f:
          f.write(fixed_code)

# generate the solutions produced by codegen
repair_humaneval_codet5_template(codet5, codet5_tokenizer, buggy_humaneval, success_humaneval_codet5, "codet5_fix_template")

  0%|          | 0/164 [00:00<?, ?it/s]

In [25]:
# note you can passing in "--i-just-wanna-run" to this command to
# recompute the results IF and ONLY IF you have made some updates to each solution file :)
# you may need to pass in (yes Y | command) on colab
!yes Y | evalplus.evaluate --dataset humaneval --samples codet5_fix_template --i-just-wanna-run

Load from ground-truth from /root/.cache/evalplus/84f4b93a1270b492e4c54d5212da7a5b.pkl
Reading samples...
164it [00:00, 3343.00it/s]
100% 164/164 [00:10<00:00, 15.92it/s]
codet5_fix_template/eval_results.json already exists. Press [Y/N] to overwrite or exit...
Backup codet5_fix_template/eval_results.json to codet5_fix_template/eval_results.json.bak.bak
Base
{'pass@1': np.float64(0.054878048780487805)}
Base + Extra
{'pass@1': np.float64(0.054878048780487805)}
